<a href="https://colab.research.google.com/github/RepnaiaE/DataScience_Emotion-recognition-methods/blob/main/Lab7_Ensembles_of_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ансамбли моделей/Ensembles of models

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Загрузка данных/Loading Data

Загружаем данные из ячеек в массивы признаков для каждого набора из предыдущей работы

In [ ]:
import pandas as pd

negative = pd.read_csv('/content/drive/MyDrive/Распознавание эмоций/negative.csv', sep=';',
                       names=['id', 'tdate', 'tname', 'ttext', 'ttype', 'trep', 'trtv',
                              'tfav', 'tstcount', 'tfol', 'tfrien', 'listcount'])
negative.head()

,id,tdate,tname,ttext,ttype,trep,trtv,tfav,tstcount,tfol,tfrien,listcount
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0


In [ ]:
positive = pd.read_csv('/content/drive/MyDrive/Распознавание эмоций/positive.csv', sep=';',
                       names=['id', 'tdate', 'tname', 'ttext', 'ttype', 'trep', 'trtv',
                              'tfav', 'tstcount', 'tfol', 'tfrien', 'listcount'])
positive.head()

,id,tdate,tname,ttext,ttype,trep,trtv,tfav,tstcount,tfol,tfrien,listcount
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1


In [ ]:
DATA_LEN = 15000

data = pd.concat([positive.head(DATA_LEN), negative.head(DATA_LEN)])
data.loc[(data.ttype < 0), ('ttype')] = 0
data

,id,tdate,tname,ttext,ttype,trep,trtv,tfav,tstcount,tfol,tfrien,listcount
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1
...,...,...,...,...,...,...,...,...,...,...,...,...
14995,411052214937874432,1386837459,nataliyatm,"@kosaya_lisa я бы была рада морозу больше, чем...",0,0,0,0,7886,62,100,0
14996,411052217857474560,1386837460,vynujapyzm,Мне противно смотреть на логи нашего провайдер...,0,0,0,0,281,264,256,0
14997,411052251843936256,1386837468,gubapikahacu,"RT @fupefodicita: ""Большинство россиян по-преж...",0,0,1,0,853,273,257,0
14998,411052507985879040,1386837529,cyberr98,@Yulia_Demchuk @Leno_ok1998 давайте в школу мн...,0,0,0,0,678,54,38,0


In [ ]:
X=data['ttext'].values.tolist()
y=data['ttype'].values.tolist()

# Обучение языковой модели/Language model training

In [ ]:
from string import punctuation
import nltk
nltk.download('punkt')

import nltk.corpus
import nltk.tokenize

punctuations = list(punctuation)
punkt = ['``','...',"''",'«','»','…','”','”','“','-','–','..']
punctuations.extend(punkt)

def tokenize(text):
    text_token = nltk.word_tokenize(text)
    text_word=[x.lower() for x in text_token if x not in punctuations and x.isalpha]
    return text_word

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
X_token = [tokenize(t) for t in X]

In [ ]:
VECTOR_SIZE = 300

In [ ]:
!pip install --upgrade gensim

In [ ]:
from gensim.models import Word2Vec

w2v_model = Word2Vec(sentences=X_token, vector_size=VECTOR_SIZE, window=5, min_count=1, workers=4)

In [ ]:
w2v_model.build_vocab(X_token)

In [ ]:
w2v_model.train(X_token, total_examples=w2v_model.corpus_count,
                epochs=10, report_delay=1)
w2v_model.init_sims(replace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
import numpy as np
import gensim

def word_averaging(w2v_model, words):
    mean = []

    for word in words:
        m = w2v_model.wv[word]
        mean.append(m)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

In [ ]:
def word_averaging_list(model, text_list):
    return np.vstack([word_averaging(model, comment_text) for comment_text in text_list])

In [ ]:
X = word_averaging_list(w2v_model, X_token)
X.shape

(30000, 300)

In [ ]:
from sklearn.model_selection import train_test_split

X, X_test, y, y_test = train_test_split(X, y, test_size=0.2)

# Ансамбли/Ensembles

In [ ]:
NUM_CLASSES = 2

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import StackingClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

def try_ensemble(X, y, X_test, y_test, n_estimators=20):
    estimators = {'DecisionTreeClassifier': DecisionTreeClassifier(max_depth=7)}
    for name, be in estimators.items():
        clf = AdaBoostClassifier(base_estimator=be, n_estimators=n_estimators, random_state=10)
        clf.fit(X, y)
        predicted = clf.predict(X_test)

        print("ADA BOOST")
        print(f"estimator: {name}")
        print(classification_report(y_test, predicted))
        print(confusion_matrix(y_test, predicted))

    print('-' * 50)
    estimators = {'DecisionTreeClassifier': DecisionTreeClassifier(max_depth=7),
                  'KNeighborsClassifier': KNeighborsClassifier()}
    for name, be in estimators.items():
        clf = BaggingClassifier(base_estimator=be, n_estimators=n_estimators, random_state=10)
        clf.fit(X, y)
        predicted = clf.predict(X_test)

        print("BAGGING")
        print(f"estimator: {name}")
        print(classification_report(y_test, predicted))
        print(confusion_matrix(y_test, predicted))

    print('-' * 50)

    estimators = {'DecisionTreeClassifier': DecisionTreeClassifier(max_depth=7),
                  'RandomForestClassifier': RandomForestClassifier(n_estimators=n_estimators, random_state=10),
                  'KNeighborsClassifier': KNeighborsClassifier()}
    for name, be in estimators.items():
        ests = [
        ('rf', be),
        ('svr', make_pipeline(StandardScaler(),
                              LinearSVC(random_state=10)))]
        clf = StackingClassifier(
        estimators=ests, final_estimator=LogisticRegression())
        clf.fit(X, y)
        predicted = clf.predict(X_test)

        print("STACKING")
        print(f"estimator: {name}")
        print(classification_report(y_test, predicted))
        print(confusion_matrix(y_test, predicted))

In [ ]:
try_ensemble(X, y, X_test, y_test, 40)

ADA BOOST
estimator: DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.60      0.59      0.60      2998
           1       0.60      0.61      0.60      3002

    accuracy                           0.60      6000
   macro avg       0.60      0.60      0.60      6000
weighted avg       0.60      0.60      0.60      6000

[[1782 1216]
 [1182 1820]]
--------------------------------------------------
BAGGING
estimator: DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.65      0.64      0.64      2998
           1       0.64      0.65      0.65      3002

    accuracy                           0.64      6000
   macro avg       0.64      0.64      0.64      6000
weighted avg       0.64      0.64      0.64      6000

[[1910 1088]
 [1051 1951]]
BAGGING
estimator: KNeighborsClassifier
              precision    recall  f1-score   support

           0       0.64      0.49      0.56      2998
      

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

STACKING
estimator: DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.62      0.63      0.63      2998
           1       0.63      0.62      0.62      3002

    accuracy                           0.62      6000
   macro avg       0.62      0.62      0.62      6000
weighted avg       0.62      0.62      0.62      6000

[[1885 1113]
 [1139 1863]]


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

STACKING
estimator: RandomForestClassifier
              precision    recall  f1-score   support

           0       0.65      0.67      0.66      2998
           1       0.66      0.64      0.65      3002

    accuracy                           0.66      6000
   macro avg       0.66      0.66      0.66      6000
weighted avg       0.66      0.66      0.66      6000

[[2019  979]
 [1085 1917]]


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

STACKING
estimator: KNeighborsClassifier
              precision    recall  f1-score   support

           0       0.63      0.64      0.64      2998
           1       0.64      0.63      0.63      3002

    accuracy                           0.63      6000
   macro avg       0.63      0.63      0.63      6000
weighted avg       0.63      0.63      0.63      6000

[[1918 1080]
 [1114 1888]]


In [ ]:
try_ensemble(X, y, X_test, y_test, 60)

ADA BOOST
estimator: DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.59      0.60      0.60      2998
           1       0.60      0.59      0.59      3002

    accuracy                           0.59      6000
   macro avg       0.60      0.60      0.59      6000
weighted avg       0.60      0.59      0.59      6000

[[1786 1212]
 [1218 1784]]
--------------------------------------------------
BAGGING
estimator: DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.64      0.64      0.64      2998
           1       0.64      0.65      0.64      3002

    accuracy                           0.64      6000
   macro avg       0.64      0.64      0.64      6000
weighted avg       0.64      0.64      0.64      6000

[[1913 1085]
 [1059 1943]]
BAGGING
estimator: KNeighborsClassifier
              precision    recall  f1-score   support

           0       0.64      0.49      0.56      2998
      

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

STACKING
estimator: DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.62      0.63      0.63      2998
           1       0.63      0.62      0.62      3002

    accuracy                           0.63      6000
   macro avg       0.63      0.63      0.63      6000
weighted avg       0.63      0.63      0.63      6000

[[1889 1109]
 [1136 1866]]


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

STACKING
estimator: RandomForestClassifier
              precision    recall  f1-score   support

           0       0.66      0.67      0.66      2998
           1       0.66      0.65      0.66      3002

    accuracy                           0.66      6000
   macro avg       0.66      0.66      0.66      6000
weighted avg       0.66      0.66      0.66      6000

[[2017  981]
 [1058 1944]]


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

STACKING
estimator: KNeighborsClassifier
              precision    recall  f1-score   support

           0       0.63      0.64      0.64      2998
           1       0.64      0.63      0.63      3002

    accuracy                           0.63      6000
   macro avg       0.63      0.63      0.63      6000
weighted avg       0.63      0.63      0.63      6000

[[1918 1080]
 [1114 1888]]


In [ ]:
try_ensemble(X, y, X_test, y_test, 20)

ADA BOOST
estimator: DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.60      0.58      0.59      2998
           1       0.59      0.61      0.60      3002

    accuracy                           0.60      6000
   macro avg       0.60      0.60      0.60      6000
weighted avg       0.60      0.60      0.60      6000

[[1735 1263]
 [1162 1840]]
--------------------------------------------------
BAGGING
estimator: DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.65      0.63      0.64      2998
           1       0.64      0.66      0.65      3002

    accuracy                           0.64      6000
   macro avg       0.64      0.64      0.64      6000
weighted avg       0.64      0.64      0.64      6000

[[1876 1122]
 [1030 1972]]
BAGGING
estimator: KNeighborsClassifier
              precision    recall  f1-score   support

           0       0.64      0.49      0.56      2998
      

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

STACKING
estimator: DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.62      0.63      0.63      2998
           1       0.63      0.62      0.62      3002

    accuracy                           0.63      6000
   macro avg       0.63      0.63      0.63      6000
weighted avg       0.63      0.63      0.63      6000

[[1891 1107]
 [1140 1862]]


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

STACKING
estimator: RandomForestClassifier
              precision    recall  f1-score   support

           0       0.65      0.67      0.66      2998
           1       0.66      0.64      0.65      3002

    accuracy                           0.65      6000
   macro avg       0.65      0.65      0.65      6000
weighted avg       0.65      0.65      0.65      6000

[[1995 1003]
 [1080 1922]]


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

STACKING
estimator: KNeighborsClassifier
              precision    recall  f1-score   support

           0       0.63      0.64      0.64      2998
           1       0.64      0.63      0.63      3002

    accuracy                           0.63      6000
   macro avg       0.63      0.63      0.63      6000
weighted avg       0.63      0.63      0.63      6000

[[1918 1080]
 [1114 1888]]


In [ ]:
try_ensemble(X, y, X_test, y_test, 100)

ADA BOOST
estimator: DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.60      0.59      0.59      2998
           1       0.59      0.60      0.60      3002

    accuracy                           0.59      6000
   macro avg       0.60      0.59      0.59      6000
weighted avg       0.60      0.59      0.59      6000

[[1766 1232]
 [1198 1804]]
--------------------------------------------------
BAGGING
estimator: DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.65      0.63      0.64      2998
           1       0.64      0.65      0.65      3002

    accuracy                           0.64      6000
   macro avg       0.64      0.64      0.64      6000
weighted avg       0.64      0.64      0.64      6000

[[1903 1095]
 [1045 1957]]
BAGGING
estimator: KNeighborsClassifier
              precision    recall  f1-score   support

           0       0.64      0.49      0.55      2998
      

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

STACKING
estimator: DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.62      0.63      0.63      2998
           1       0.63      0.62      0.62      3002

    accuracy                           0.62      6000
   macro avg       0.63      0.63      0.62      6000
weighted avg       0.63      0.62      0.62      6000

[[1885 1113]
 [1137 1865]]


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

STACKING
estimator: RandomForestClassifier
              precision    recall  f1-score   support

           0       0.67      0.69      0.68      2998
           1       0.68      0.66      0.67      3002

    accuracy                           0.67      6000
   macro avg       0.67      0.67      0.67      6000
weighted avg       0.67      0.67      0.67      6000

[[2055  943]
 [1033 1969]]


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

STACKING
estimator: KNeighborsClassifier
              precision    recall  f1-score   support

           0       0.63      0.64      0.64      2998
           1       0.64      0.63      0.63      3002

    accuracy                           0.63      6000
   macro avg       0.63      0.63      0.63      6000
weighted avg       0.63      0.63      0.63      6000

[[1918 1080]
 [1114 1888]]


# Предобученная языковая модель/Pretrained language model

In [ ]:
model_url = 'http://vectors.nlpl.eu/repository/20/180.zip'

In [ ]:
import gensim

model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Распознавание эмоций/model.bin', binary=True)

In [ ]:
import pickle

data = open('/content/drive/MyDrive/Распознавание эмоций/sentiment_POS', 'rb')
loaded_data = pickle.load(data)
loaded_data

,ttext,ttype,text_pos,X
0,на работе был полный пиддес :| и так каждое за...,0,на_ADP работа_NOUN быть_AUX полный_ADJ пиддо_N...,"[на_ADP, работа_NOUN, быть_AUX, полный_ADJ, пи..."
1,"Коллеги сидят рубятся в Urban terror, а я из-з...",0,коллега_NOUN сидеть_VERB рубиться_VERB в_ADP u...,"[коллега_NOUN, сидеть_VERB, рубиться_VERB, в_A..."
2,@elina_4post как говорят обещаного три года жд...,0,@elina4post_X как_SCONJ говорить_VERB обещаный...,"[@elina4post_X, как_SCONJ, говорить_VERB, обещ..."
3,"Желаю хорошего полёта и удачной посадки,я буду...",0,желать_VERB хороший_ADJ полет_NOUN и_CCONJ уда...,"[желать_VERB, хороший_ADJ, полет_NOUN, и_CCONJ..."
4,"Обновил за каким-то лешим surf, теперь не рабо...",0,обновить_VERB за_ADP какой-то_ADJ лешимый_ADJ ...,"[обновить_VERB, за_ADP, какой-то_ADJ, лешимый_..."
...,...,...,...,...
1999,уаааа)))))) сейчас около меня прошел паренек с...,1,уааа_NOUN сейчас_ADV около_ADP я_PRON проходит...,"[уааа_NOUN, сейчас_ADV, около_ADP, я_PRON, про..."
2000,За монгол коментийн 2 оо vзэж жаахан хэрэгт ду...,1,за_ADP монгол_NOUN коментийн_NOUN x_NUM оо_ADV...,"[за_ADP, монгол_NOUN, коментийн_NOUN, x_NUM, о..."
2001,"""@AdelKhalilova: @Nastasseya @AnuTochkaSavina ...",1,@adelkhalilova_PROPN @nastasseya_PROPN @anutoc...,"[@adelkhalilova_PROPN, @nastasseya_PROPN, @anu..."
2002,"Дим, ты помогаешь мне, я тебе, все взаимно, вс...",1,дима_PROPN ты_PRON помогать_VERB я_PRON я_PRON...,"[дима_PROPN, ты_PRON, помогать_VERB, я_PRON, я..."


In [ ]:
import pandas as pd
import numpy as np

def word_averaging(model, words):
    all_words, mean = set(), []

    for word in words:
        if word in model.key_to_index.keys():
            mean.append(model[model.key_to_index[word]])
            all_words.add(model.key_to_index[word])
        else:
            mean.append(np.zeros(w2v_model.vector_size))

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

In [ ]:
def word_averaging_list(model, text_list):
    return np.vstack([word_averaging(model, comment_text) for comment_text in text_list ])

In [ ]:
y = loaded_data['ttype'].values.tolist()

X=loaded_data['X']
X = word_averaging_list(model, X)
X.shape

(2004, 300)

In [ ]:
from sklearn.model_selection import train_test_split

X, X_test, y, y_test = train_test_split(X, y, test_size=0.2)

# Ансамбли/Ensembles

In [ ]:
try_ensemble(X, y, X_test, y_test, 40)

ADA BOOST
estimator: DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.57      0.54      0.55       198
           1       0.57      0.60      0.58       203

    accuracy                           0.57       401
   macro avg       0.57      0.57      0.57       401
weighted avg       0.57      0.57      0.57       401

[[107  91]
 [ 82 121]]
--------------------------------------------------
BAGGING
estimator: DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.67      0.63      0.65       198
           1       0.66      0.70      0.68       203

    accuracy                           0.67       401
   macro avg       0.67      0.67      0.67       401
weighted avg       0.67      0.67      0.67       401

[[125  73]
 [ 61 142]]
BAGGING
estimator: KNeighborsClassifier
              precision    recall  f1-score   support

           0       0.55      0.42      0.48       198
           1  

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

STACKING
estimator: DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.65      0.65      0.65       198
           1       0.66      0.67      0.66       203

    accuracy                           0.66       401
   macro avg       0.66      0.66      0.66       401
weighted avg       0.66      0.66      0.66       401

[[128  70]
 [ 68 135]]


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

STACKING
estimator: RandomForestClassifier
              precision    recall  f1-score   support

           0       0.67      0.66      0.67       198
           1       0.67      0.68      0.68       203

    accuracy                           0.67       401
   macro avg       0.67      0.67      0.67       401
weighted avg       0.67      0.67      0.67       401

[[131  67]
 [ 64 139]]


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


STACKING
estimator: KNeighborsClassifier
              precision    recall  f1-score   support

           0       0.64      0.67      0.66       198
           1       0.66      0.64      0.65       203

    accuracy                           0.65       401
   macro avg       0.65      0.65      0.65       401
weighted avg       0.65      0.65      0.65       401

[[133  65]
 [ 74 129]]


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [ ]:
try_ensemble(X, y, X_test, y_test, 60)

ADA BOOST
estimator: DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.58      0.62      0.60       198
           1       0.60      0.56      0.58       203

    accuracy                           0.59       401
   macro avg       0.59      0.59      0.59       401
weighted avg       0.59      0.59      0.59       401

[[122  76]
 [ 89 114]]
--------------------------------------------------
BAGGING
estimator: DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.65      0.64      0.64       198
           1       0.65      0.66      0.66       203

    accuracy                           0.65       401
   macro avg       0.65      0.65      0.65       401
weighted avg       0.65      0.65      0.65       401

[[126  72]
 [ 69 134]]
BAGGING
estimator: KNeighborsClassifier
              precision    recall  f1-score   support

           0       0.56      0.42      0.48       198
           1  

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

STACKING
estimator: DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.65      0.65      0.65       198
           1       0.66      0.67      0.66       203

    accuracy                           0.66       401
   macro avg       0.66      0.66      0.66       401
weighted avg       0.66      0.66      0.66       401

[[128  70]
 [ 68 135]]


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

STACKING
estimator: RandomForestClassifier
              precision    recall  f1-score   support

           0       0.66      0.66      0.66       198
           1       0.67      0.67      0.67       203

    accuracy                           0.66       401
   macro avg       0.66      0.66      0.66       401
weighted avg       0.66      0.66      0.66       401

[[131  67]
 [ 68 135]]


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


STACKING
estimator: KNeighborsClassifier
              precision    recall  f1-score   support

           0       0.64      0.67      0.66       198
           1       0.66      0.64      0.65       203

    accuracy                           0.65       401
   macro avg       0.65      0.65      0.65       401
weighted avg       0.65      0.65      0.65       401

[[133  65]
 [ 74 129]]


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [ ]:
try_ensemble(X, y, X_test, y_test, 20)

ADA BOOST
estimator: DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.55      0.52      0.53       198
           1       0.56      0.60      0.58       203

    accuracy                           0.56       401
   macro avg       0.56      0.56      0.56       401
weighted avg       0.56      0.56      0.56       401

[[102  96]
 [ 82 121]]
--------------------------------------------------
BAGGING
estimator: DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.66      0.64      0.65       198
           1       0.66      0.68      0.67       203

    accuracy                           0.66       401
   macro avg       0.66      0.66      0.66       401
weighted avg       0.66      0.66      0.66       401

[[126  72]
 [ 65 138]]
BAGGING
estimator: KNeighborsClassifier
              precision    recall  f1-score   support

           0       0.55      0.41      0.47       198
           1  

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

STACKING
estimator: DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.66      0.65      0.65       198
           1       0.66      0.67      0.67       203

    accuracy                           0.66       401
   macro avg       0.66      0.66      0.66       401
weighted avg       0.66      0.66      0.66       401

[[128  70]
 [ 67 136]]


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

STACKING
estimator: RandomForestClassifier
              precision    recall  f1-score   support

           0       0.65      0.67      0.66       198
           1       0.66      0.65      0.65       203

    accuracy                           0.66       401
   macro avg       0.66      0.66      0.66       401
weighted avg       0.66      0.66      0.66       401

[[132  66]
 [ 72 131]]


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


STACKING
estimator: KNeighborsClassifier
              precision    recall  f1-score   support

           0       0.64      0.67      0.66       198
           1       0.66      0.64      0.65       203

    accuracy                           0.65       401
   macro avg       0.65      0.65      0.65       401
weighted avg       0.65      0.65      0.65       401

[[133  65]
 [ 74 129]]


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [ ]:
try_ensemble(X, y, X_test, y_test, 100)

ADA BOOST
estimator: DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.60      0.58      0.59       198
           1       0.60      0.63      0.61       203

    accuracy                           0.60       401
   macro avg       0.60      0.60      0.60       401
weighted avg       0.60      0.60      0.60       401

[[114  84]
 [ 76 127]]
--------------------------------------------------
BAGGING
estimator: DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.67      0.64      0.65       198
           1       0.66      0.69      0.68       203

    accuracy                           0.67       401
   macro avg       0.67      0.67      0.67       401
weighted avg       0.67      0.67      0.67       401

[[127  71]
 [ 63 140]]
BAGGING
estimator: KNeighborsClassifier
              precision    recall  f1-score   support

           0       0.57      0.43      0.49       198
           1  

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

STACKING
estimator: DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.65      0.64      0.65       198
           1       0.66      0.67      0.66       203

    accuracy                           0.65       401
   macro avg       0.65      0.65      0.65       401
weighted avg       0.65      0.65      0.65       401

[[127  71]
 [ 68 135]]


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

STACKING
estimator: RandomForestClassifier
              precision    recall  f1-score   support

           0       0.68      0.66      0.67       198
           1       0.67      0.69      0.68       203

    accuracy                           0.68       401
   macro avg       0.68      0.68      0.68       401
weighted avg       0.68      0.68      0.68       401

[[130  68]
 [ 62 141]]


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


STACKING
estimator: KNeighborsClassifier
              precision    recall  f1-score   support

           0       0.64      0.67      0.66       198
           1       0.66      0.64      0.65       203

    accuracy                           0.65       401
   macro avg       0.65      0.65      0.65       401
weighted avg       0.65      0.65      0.65       401

[[133  65]
 [ 74 129]]


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


# Выводы/Conclusion

Было обучено множество моделей на основе Net (описана в пункте Перебор параметров) с перебором параметров для обучения. Обучение проводилось на признаках, выделенных с помощью обученной модели и предобученной.

Many Net-based models were trained (described in Parametric Brute Force) with parameter brute force training. The training was carried out on features extracted using the trained model and the pre-trained one.

## Результаты предобученной модели/Pretrainied model results

| Features | Ensemble | n_estimators | Model | F1 score |
|----------|----------|--------------|-------|----------|
| pretrained w2v | STACKING | 100 | RandomForestClassifier | 0.68 |
| pretrained w2v | STACKING | 40 | RandomForestClassifier | 0.67 |
| pretrained w2v | BAGGING | 20 | DecisionTreeClassifier | 0.66 |

## Результаты обученной модели/Trainied model results

| Features | Ensemble | n_estimators | Model | F1 score |
|----------|----------|--------------|-------|----------|
| w2v | STACKING | 100 | RandomForestClassifier | 0.67 |
| w2v | STACKING | 60 | RandomForestClassifier | 0.66 |
| w2v | BAGGING | 100 | DecisionTreeClassifier | 0.64 |